In [1]:
! pip install psycopg2 sqlalchemy python-dotenv pandas

  Using cached psycopg2-2.9.10-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached greenlet-3.1.1-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached psycopg2-2.9.10-cp311-cp311-win_amd64.whl (1.2 MB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 1.9 MB/s eta 0:00:01
   -------------- ------------------------- 0.8/2.1 MB 1.8 MB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 2.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 2.1 MB/s eta 0:00

In [2]:
import os 
from dotenv import load_dotenv # for environment variables
import psycopg2 # for connecting to postgres
from sqlalchemy import create_engine  
import pandas as pd

# load environment variables from .env file
load_dotenv()

# access the environment variables
db_user = os.environ.get('POSTGRES_USER')
db_password = os.environ.get('POSTGRES_PASSWORD')
db_name = os.environ.get('POSTGRES_DB')
db_host = 'localhost'
db_port = '5432'

In [3]:
print(db_user, db_password, db_name, db_host, db_port)

postgres example postgres localhost 5432


In [4]:
try:
    # Connect to the database
    conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        dbname=db_name,
        user=db_user,
        password=db_password
    )
    print("Connected to the database successfully!")
except psycopg2.Error as e: 
    print(f"Error connecting to the database: {e}")

Connected to the database successfully!


In [5]:
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')  

In [6]:
csv_files_directory = './data'
if not os.path.exists(csv_files_directory): # check if the directory exists
    print(f"Directory '{csv_files_directory}' does not exist.")
    
csv_files = [f for f in os.listdir(csv_files_directory) if f.endswith('.csv')] # get all csv files in the directory

In [7]:
csv_files

['CUST_AZ12.csv',
 'cust_info.csv',
 'LOC_A101.csv',
 'prd_info.csv',
 'PX_CAT_G1V2.csv',
 'sales_details.csv']

In [8]:
def load_csv_to_postgresql(csv_files, table_name):
   try: 
        df = pd.read_csv(csv_files) # read csv file
        columns = ",".join([f"{cols} TEXT" for cols in df.columns]) # Read coulumns give text data type and join # to string
        print(columns)
        create_table_query = f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
                {columns}
            )
            """ # create table
            
        print(f"Creating Table '{table_name}' with query: {create_table_query}")  # print create table query
        
        with conn.cursor() as cursor: # we don't have to write exit using ' with '
            cursor.execute(create_table_query) # execute create table query
            conn.commit() # commit changes
            
        df.to_sql(table_name, engine, if_exists='replace', index=False, method='multi', chunksize=1000) # insert data into table
        print(f"Data from {csv_files} has been successfully loaded into {table_name}")
        
   except Exception as e:
       print(f"Error loading data from {csv_files} into {table_name}: {e}")    
       
for csv_file in csv_files: # loop through csv files
    csv_file_path = os.path.join(csv_files_directory, csv_file) # get csv file path
    table_name = os.path.splitext(csv_file)[0] # get table name
    
    load_csv_to_postgresql(csv_file_path, table_name) # call function
    
conn.close()
print("PostgreSQL connection closed")        

CID TEXT,BDATE TEXT,GEN TEXT
Creating Table 'CUST_AZ12' with query: 
            CREATE TABLE IF NOT EXISTS CUST_AZ12 (
                CID TEXT,BDATE TEXT,GEN TEXT
            )
            
Data from ./data\CUST_AZ12.csv has been successfully loaded into CUST_AZ12
cst_id TEXT,cst_key TEXT,cst_firstname TEXT,cst_lastname TEXT,cst_marital_status TEXT,cst_gndr TEXT,cst_create_date TEXT
Creating Table 'cust_info' with query: 
            CREATE TABLE IF NOT EXISTS cust_info (
                cst_id TEXT,cst_key TEXT,cst_firstname TEXT,cst_lastname TEXT,cst_marital_status TEXT,cst_gndr TEXT,cst_create_date TEXT
            )
            
Data from ./data\cust_info.csv has been successfully loaded into cust_info
CID TEXT,CNTRY TEXT
Creating Table 'LOC_A101' with query: 
            CREATE TABLE IF NOT EXISTS LOC_A101 (
                CID TEXT,CNTRY TEXT
            )
            
Data from ./data\LOC_A101.csv has been successfully loaded into LOC_A101
prd_id TEXT,prd_key TEXT,prd_nm TEXT,